In [10]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

from helpers import count_unique_words, count_unique_ngrams, \
            build_unique_ngrams, create_sentence_vectors, create_sentence_vectors_submission

import sys

import tensorflow as tf
from tensorflow import keras

import gensim   # Not sure whether it is better to use gensim or tensorflow :/
import logging
from gensim.models.phrases import Phrases, Phraser

import multiprocessing

from gensim.models import Word2Vec
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
w2v_model = Word2Vec.load("models/w2v_model_epochs_5_win_5_sg_200.model")

2019-12-15 15:20:49,501 : INFO : loading Word2Vec object from models/w2v_model_epochs_5_win_5_sg_200.model
2019-12-15 15:20:50,198 : INFO : loading wv recursively from models/w2v_model_epochs_5_win_5_sg_200.model.wv.* with mmap=None
2019-12-15 15:20:50,198 : INFO : loading vectors from models/w2v_model_epochs_5_win_5_sg_200.model.wv.vectors.npy with mmap=None
2019-12-15 15:20:50,858 : INFO : setting ignored attribute vectors_norm to None
2019-12-15 15:20:50,860 : INFO : loading vocabulary recursively from models/w2v_model_epochs_5_win_5_sg_200.model.vocabulary.* with mmap=None
2019-12-15 15:20:50,861 : INFO : loading trainables recursively from models/w2v_model_epochs_5_win_5_sg_200.model.trainables.* with mmap=None
2019-12-15 15:20:50,861 : INFO : loading syn1neg from models/w2v_model_epochs_5_win_5_sg_200.model.trainables.syn1neg.npy with mmap=None
2019-12-15 15:20:51,625 : INFO : setting ignored attribute cum_table to None
2019-12-15 15:20:51,626 : INFO : loaded models/w2v_model_epo

In [12]:
len(w2v_model.wv.vocab)

452520

In [13]:
word_vector_size = w2v_model.wv.vectors.shape[1]

In [3]:
# This is unfortunately very high memory consumptive

# Test here is not the unlabelled testset, but rather the 0.8 split of the original training set

sentence_train_x = np.load("np_arrays/sentences_train_x_0_8.npy")
sentence_train_y = np.load("np_arrays/sentences_train_y_0_8.npy")

sentence_test_x = np.load("np_arrays/sentences_test_x_0_2.npy")
sentence_test_y = np.load("np_arrays/sentences_test_y_0_2.npy")

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint
# now perform training on the new features vectors.

# Build a "deep" neural network with 2 hidden layers. When we see that it somehow works,
# we can start doing some cross validation on it.

for layer_size in range(40, 101, 5):
    
    print("\n\nStarting with number of layers: {}".format(layer_size))
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy']
                  )

    overfitting_occurrences = 0
    best_accuracy = 0

    filepath="models/weights_2_layers_{}_big".format(layer_size)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    model.fit(x=sentence_train_x,
              y=sentence_train_y,
              validation_data=(sentence_test_x,  sentence_test_y),
              callbacks=callbacks_list,
              epochs=50,
              use_multiprocessing=True,
              batch_size=128)
        




Starting with number of layers: 40
Instructions for updating:
Colocations handled automatically by placer.


2019-12-14 21:38:06,896 : WARNING : From /home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Train on 2000000 samples, validate on 500000 samples
Instructions for updating:
Use tf.cast instead.


2019-12-14 21:38:07,018 : WARNING : From /home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/50
1999872/2000000 [============================>.] - ETA: 0s - loss: 0.4244 - acc: 0.7958
Epoch 00001: val_acc improved from -inf to 0.80229, saving model to models/weights_2_layers_40_big
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4244 - acc: 0.7958 - val_loss: 0.4144 - val_acc: 0.8023
Epoch 2/50
1997440/2000000 [============================>.] - ETA: 0s - loss: 0.4086 - acc: 0.8052
Epoch 00002: val_acc improved from 0.80229 to 0.80303, saving model to models/weights_2_layers_40_big
2000000/2000000 [==============================] - 27s 14us/sample - loss: 0.4086 - acc: 0.8052 - val_loss: 0.4111 - val_acc: 0.8030
Epoch 3/50
1997568/2000000 [============================>.] - ETA: 0s - loss: 0.4036 - acc: 0.8080
Epoch 00003: val_acc improved from 0.80303 to 0.80846, saving model to models/weights_2_layers_40_big
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4035 - acc: 0.8080 - val_loss: 0.4028 - val_acc: 0.8085
E

...

Epoch 48/50
1998080/2000000 [============================>.] - ETA: 0s - loss: 0.3697 - acc: 0.8263
Epoch 00048: val_acc did not improve from 0.81761
2000000/2000000 [==============================] - 31s 15us/sample - loss: 0.3697 - acc: 0.8263 - val_loss: 0.3889 - val_acc: 0.8172
Epoch 49/50
1997440/2000000 [============================>.] - ETA: 0s - loss: 0.3696 - acc: 0.8264
Epoch 00049: val_acc did not improve from 0.81761
2000000/2000000 [==============================] - 31s 15us/sample - loss: 0.3696 - acc: 0.8264 - val_loss: 0.3889 - val_acc: 0.8165
Epoch 50/50
1999360/2000000 [============================>.] - ETA: 0s - loss: 0.3695 - acc: 0.8262
Epoch 00050: val_acc did not improve from 0.81761
2000000/2000000 [==============================] - 31s 15us/sample - loss: 0.3695 - acc: 0.8263 - val_loss: 0.3909 - val_acc: 0.8151


Starting with number of layers: 90
Train on 2000000 samples, validate on 500000 samples
Epoch 1/50
1998976/2000000 [============================>.] -

Epoch 25/50
1998848/2000000 [============================>.] - ETA: 0s - loss: 0.3724 - acc: 0.8246
Epoch 00025: val_acc did not improve from 0.81711
2000000/2000000 [==============================] - 33s 16us/sample - loss: 0.3724 - acc: 0.8246 - val_loss: 0.3871 - val_acc: 0.8168
Epoch 26/50
1997440/2000000 [============================>.] - ETA: 0s - loss: 0.3721 - acc: 0.8250
Epoch 00026: val_acc did not improve from 0.81711
2000000/2000000 [==============================] - 33s 16us/sample - loss: 0.3721 - acc: 0.8250 - val_loss: 0.3875 - val_acc: 0.8166
Epoch 27/50
1996672/2000000 [============================>.] - ETA: 0s - loss: 0.3718 - acc: 0.8249
Epoch 00027: val_acc did not improve from 0.81711
2000000/2000000 [==============================] - 33s 16us/sample - loss: 0.3718 - acc: 0.8249 - val_loss: 0.3873 - val_acc: 0.8166
Epoch 28/50
1999744/2000000 [============================>.] - ETA: 0s - loss: 0.3715 - acc: 0.8253
Epoch 00028: val_acc did not improve from 0.81711
2

Epoch 3/50
1997696/2000000 [============================>.] - ETA: 0s - loss: 0.3949 - acc: 0.8126
Epoch 00003: val_acc improved from 0.81056 to 0.81151, saving model to models/weights_2_layers_95_big
2000000/2000000 [==============================] - 33s 17us/sample - loss: 0.3949 - acc: 0.8126 - val_loss: 0.3964 - val_acc: 0.8115
Epoch 4/50
1999360/2000000 [============================>.] - ETA: 0s - loss: 0.3909 - acc: 0.8148
Epoch 00004: val_acc improved from 0.81151 to 0.81352, saving model to models/weights_2_layers_95_big
2000000/2000000 [==============================] - 33s 17us/sample - loss: 0.3909 - acc: 0.8148 - val_loss: 0.3921 - val_acc: 0.8135
Epoch 5/50
1998336/2000000 [============================>.] - ETA: 0s - loss: 0.3881 - acc: 0.8165
Epoch 00005: val_acc improved from 0.81352 to 0.81388, saving model to models/weights_2_layers_95_big
2000000/2000000 [==============================] - 33s 17us/sample - loss: 0.3881 - acc: 0.8165 - val_loss: 0.3915 - val_acc: 0.813

Epoch 8/50
1998592/2000000 [============================>.] - ETA: 0s - loss: 0.3819 - acc: 0.8195
Epoch 00008: val_acc did not improve from 0.81500
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.3819 - acc: 0.8195 - val_loss: 0.3921 - val_acc: 0.8141
Epoch 9/50
1997440/2000000 [============================>.] - ETA: 0s - loss: 0.3807 - acc: 0.8203
Epoch 00009: val_acc improved from 0.81500 to 0.81539, saving model to models/weights_2_layers_100_big
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.3807 - acc: 0.8203 - val_loss: 0.3899 - val_acc: 0.8154
Epoch 10/50
1999488/2000000 [============================>.] - ETA: 0s - loss: 0.3796 - acc: 0.8207
Epoch 00010: val_acc did not improve from 0.81539
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.3796 - acc: 0.8207 - val_loss: 0.3965 - val_acc: 0.8104
Epoch 11/50
1997312/2000000 [============================>.] - ETA: 0s - loss: 0.3785 - acc: 0.8215


Epoch 36/50
1996672/2000000 [============================>.] - ETA: 0s - loss: 0.3678 - acc: 0.8271
Epoch 00036: val_acc did not improve from 0.81719
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.3678 - acc: 0.8271 - val_loss: 0.3911 - val_acc: 0.8152
Epoch 37/50
1998848/2000000 [============================>.] - ETA: 0s - loss: 0.3676 - acc: 0.8272
Epoch 00037: val_acc did not improve from 0.81719
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.3676 - acc: 0.8272 - val_loss: 0.3893 - val_acc: 0.8162
Epoch 38/50
1996160/2000000 [============================>.] - ETA: 0s - loss: 0.3675 - acc: 0.8273
Epoch 00038: val_acc did not improve from 0.81719
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.3675 - acc: 0.8273 - val_loss: 0.3901 - val_acc: 0.8161
Epoch 39/50
1998464/2000000 [============================>.] - ETA: 0s - loss: 0.3673 - acc: 0.8273
Epoch 00039: val_acc did not improve from 0.81719
2

## To execute this after having trained the above model

After having decided which one is the best model to use, just load the correct model, set the parameters of the model_star so that it matches the one of the best model, and compute the results!

(The below part needs to be rewritten to take advantage of the python modules)

In [14]:
# Recreate exactl the best model
model_star = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(95, activation='relu'),
        keras.layers.Dense(95, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])

model_star.load_weights('models/weights_2_layers_95_big')

model_star.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [15]:
print(model_star)

In [16]:
# Test it on the test dataset to see if it gives the same best results.
model_star.evaluate(sentence_test_x,  sentence_test_y, verbose=2)[1]
model_star.predict(sentence_test_x)

 - 9s - loss: 0.3857 - acc: 0.8178


array([[0.8765219 , 0.12347814],
       [0.88183546, 0.11816448],
       [0.5569276 , 0.44307238],
       ...,
       [0.5233446 , 0.4766554 ],
       [0.16624893, 0.8337511 ],
       [0.9557599 , 0.04424008]], dtype=float32)

## GET the test data and predict them

In [17]:
test_df_cleaned = pd.read_pickle("dataframes/test_cleaned.pickle")

In [18]:
test_df_cleaned.head()

,sentence,label
0,sea doo pro sea scooter sport portable sea-doo...,1
1,shuck well work week can't come cheer ! oh put...,2
2,cant stay away bug thats baby,3
3,ma'am ! ! ! lol im perfectly fine contagious a...,4
4,whenever fall asleep watching tv always wake h...,5


In [19]:
sentence_submission_x = create_sentence_vectors_submission(test_df_cleaned['sentence'], word_vector_size, w2v_model)

the number of zero sentences (the sentences which have 0 words in our vocabulary) is 19


In [20]:
sentence_submission_x.shape

(10000, 200)

In [21]:
model_star.predict(sentence_submission_x)

array([[9.2009920e-01, 7.9900824e-02],
       [5.8239895e-01, 4.1760108e-01],
       [7.4870557e-01, 2.5129440e-01],
       ...,
       [9.9923396e-01, 7.6610118e-04],
       [1.6513628e-01, 8.3486372e-01],
       [9.9701947e-01, 2.9805552e-03]], dtype=float32)

In [22]:
predictions = []
for el in model_star.predict(sentence_submission_x):
    predictions.append(-1 if el[0] > el[1] else 1)

print(predictions[:10])

results = pd.DataFrame({
    "Id": test_df_cleaned['label'],
    "Prediction": predictions
})

results.head(20)

[-1, -1, -1, 1, -1, -1, -1, 1, 1, 1]


,Id,Prediction
0,1,-1
1,2,-1
2,3,-1
3,4,1
4,5,-1
5,6,-1
6,7,-1
7,8,1
8,9,1
9,10,1


In [23]:
results.to_csv('Submission.csv', index=False)